# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings

warnings.filterwarnings("ignore")

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Crew, Task

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os

openai_api_key = os.environ["OPENAI_API_KEY"]
os.environ["OPENAI_MODEL_NAME"] = "gpt-4-turbo"

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue " "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
)

In [6]:
# Agent 2: Logistics Manager
logistics_manager = Agent(
    role="Logistics Manager",
    goal=("Manage all logistics for the event " "including catering and equipmen"),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and " "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel


# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
    "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
    # Outputs the venue details as a JSON file
    agent=venue_coordinator,
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
    "equipment for an event "
    "with {expected_participants} participants "
    "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager,
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
    "aiming to engage at least"
    "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent,
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, logistics_manager, marketing_communications_agent],
    tasks=[venue_task, logistics_task, marketing_task],
    verbose=True,
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [13]:
event_details = {
    "event_topic": "Tech Innovation Conference",
    "event_description": "A gathering of tech innovators "
    "and industry leaders "
    "to explore future technologies.",
    "event_city": "San Francisco",
    "tentative_date": "2024-09-15",
    "expected_participants": 500,
    "budget": 20000,
    "venue_type": "Conference Hall",
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [14]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
Thought: I need to find a venue in San Francisco that can host a Tech Innovation Conference. First, I should search the internet for available venues in San Francisco that are suitable for hosting conferences.

Action: Search the internet
Action Input: {"search_query": "conference venues in San Francisco suitable for tech events"} 


Search results: Title: The 25 Best Event Venues in San Francisco - Bizzabo
Link: https://www.bizzabo.com/blog/san-francisco-event-venues
Snippet: San Francisco is an amazing city with exciting and unique event venues to choose from. Discover our top picks for San Francisco event venues ...
---
Title: The 16 Best Conference Venues for Rent in San Francisco, CA
Link: https://www.tagvenue.com/us/hire/conference-venues/san-francisco
Snippet: Top Conference Venues i



> Entering new CrewAgentExecutor chain...
Thought: First, I need to gather information on potential catering options suitable for an event with 500 attendees. I will start by searching the internet for highly rated catering companies in San Francisco.
Action: Search the internet
Action Input: {"search_query": "top rated catering companies in San Francisco"}Thought: To effectively market the Tech Innovation Conference and reach at least 500 potential attendees, I need to develop a comprehensive marketing strategy. This should include identifying relevant platforms for promotion, potential partnerships for broader reach, and engaging content that highlights the event's benefits. I should begin by searching for best practices in event marketing specifically tailored to tech conferences.

Action: Search the internet
Action Input: {"search_query": "best practices in marketing tech conferences"} 


Search results: Title: Top Caterers In San Francisco - PartySlate
Link: https://www.partysla

- Display the generated `venue_details.json` file.

In [17]:
import json
from pprint import pprint

with open("venue_details.json") as f:
    data = json.load(f)

pprint(data)

{'address': 'San Francisco',
 'booking_status': 'available',
 'capacity': 2000,
 'name': 'City View at METREON'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [18]:
from IPython.display import Markdown

Markdown("marketing_report.md")

## Marketing Plan for the Tech Innovation Conference

### Objective
Engage at least 500 potential attendees for the Tech Innovation Conference at City View at METREON, San Francisco.

### Understanding Your Audience
- **Surveys and Data Analysis**: Conduct online surveys and analyze data from previous events, social media, website analytics to understand demographics, interests, and factors influencing their decision to attend.

### Branding the Conference
- **Unique Selling Proposition**: Highlight unique aspects of the conference, such as cutting-edge tech discussions, renowned speakers, and networking opportunities.
- **Visual and Narrative Identity**: Develop a consistent visual theme across all promotional materials and craft compelling narratives that resonate with the target tech audience.

### Digital Marketing Strategies
- **Social Media**: Use platforms like LinkedIn, Twitter, and Facebook to promote the event. Create a unique hashtag, engage with tech groups, and run targeted ads.
- **Email Marketing**: Segment the email list and tailor messages based on past attendance and specific interests. Utilize strong calls-to-action and track open rates to adjust strategies.
- **Content Marketing**: Publish blog posts, articles, and videos that provide value and insight into the conference themes and speakers.

### Website and Registration Optimization
- **Website Usability**: Ensure the conference website is user-friendly, mobile-optimized, and SEO-friendly. Highlight key information like dates, venue, and speakers.
- **Registration Process**: Simplify the registration process, ensure security measures are in place, and provide multiple payment options.

### Leveraging Partnerships
- **Sponsors and Exhibitors**: Partner with leading tech companies and startups for financial support and broader reach. Highlight their brands in conference promotions.
- **Industry Influencers**: Collaborate with tech influencers who can promote the conference to their followers.

### Offline Marketing Tactics
- **Print Materials**: Design and distribute brochures, flyers, and posters in tech hubs, universities, and community centers.
- **Direct Mail**: Send personalized invitations to past attendees and prominent figures in the tech industry.

### Early Bird Discounts
- Offer early bird discounts to stimulate quick registrations. Set strategic deadlines to create urgency.

### Conference Content Promotion
- **Speaker Spotlights and Session Previews**: Feature key speakers and session teasers in promotions to build excitement and showcase the value of the conference.

### Engagement and Interactivity
- **Contests and Quizzes**: Run contests with free tickets as prizes and quizzes related to tech to engage potential attendees.
- **Interactive Pre-Event Content**: Host live Q&A sessions with speakers and virtual tours of the venue to increase interest and attendance.

### Conference Technology Solutions
- **Expo Logic Solutions**: Implement advanced registration and attendee tracking systems provided by Expo Logic to enhance the attendee experience and streamline event logistics.

### Call to Action
Encourage potential attendees to register early by visiting the conference website, where they can also find detailed information about the event schedule, speakers, and more.

By executing this comprehensive marketing strategy, we aim to not only meet but exceed the target of engaging 500 attendees, ensuring a successful and memorable Tech Innovation Conference.